In [1]:
import pandas as pd
import os

In [2]:
orders = pd.read_csv("../Extraction/Landing/orders/orders.csv")
orders

,order_id,customer_id,order_status,order_date,required_date,shipped_date,store_id,staff_id,Extraction_Date_Time,Source
0,1,259,4,2016-01-01,2016-01-03,2016-01-03,1,2,2024-05-03 23:35:33.118721,DB
1,2,1212,4,2016-01-01,2016-01-04,2016-01-03,2,6,2024-05-03 23:35:33.118721,DB
2,3,523,4,2016-01-02,2016-01-05,2016-01-03,2,7,2024-05-03 23:35:33.118721,DB
3,4,175,4,2016-01-03,2016-01-04,2016-01-05,1,3,2024-05-03 23:35:33.118721,DB
4,5,1324,4,2016-01-03,2016-01-06,2016-01-06,2,6,2024-05-03 23:35:33.118721,DB
...,...,...,...,...,...,...,...,...,...,...
1610,1611,6,3,2018-09-06,2018-09-06,NaN,2,7,2024-05-03 23:35:33.118721,DB
1611,1612,3,3,2018-10-21,2018-10-21,NaN,1,3,2024-05-03 23:35:33.118721,DB
1612,1613,1,3,2018-11-18,2018-11-18,NaN,2,6,2024-05-03 23:35:33.118721,DB
1613,1614,135,3,2018-11-28,2018-11-28,NaN,3,8,2024-05-03 23:35:33.118721,DB


In [3]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1615 entries, 0 to 1614
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   order_id              1615 non-null   int64 
 1   customer_id           1615 non-null   int64 
 2   order_status          1615 non-null   int64 
 3   order_date            1615 non-null   object
 4   required_date         1615 non-null   object
 5   shipped_date          1445 non-null   object
 6   store_id              1615 non-null   int64 
 7   staff_id              1615 non-null   int64 
 8   Extraction_Date_Time  1615 non-null   object
 9   Source                1615 non-null   object
dtypes: int64(5), object(5)
memory usage: 126.3+ KB


In [4]:
#fix any error in dataset before validate the data set in range 2000 t0 2024  
orders['order_date'].unique()

array(['2016-01-01', '2016-01-02', '2016-01-03', '1016-01-04',
       '1016-01-05', '1016-01-06', '2016-01-08', '2016-01-09',
       '2016-01-12', '2016-01-14', '2016-01-15', '2016-01-16',
       '2016-01-18', '2016-01-19', '2016-01-20', '2016-01-21',
       '2016-01-22', '2016-01-23', '2016-01-25', '2016-01-27',
       '2016-01-28', '2016-01-29', '2016-01-30', '2016-01-31',
       '2016-02-01', '2016-02-03', '2016-02-04', '2016-02-05',
       '2016-02-06', '2016-02-07', '2016-02-08', '2016-02-09',
       '2016-02-10', '2016-02-11', '2016-02-12', '2016-02-13',
       '2016-02-14', '2016-02-16', '2016-02-17', '2016-02-18',
       '2016-02-19', '2016-02-20', '2016-02-21', '2016-02-23',
       '2016-02-25', '2016-02-26', '2016-02-27', '2016-02-28',
       '2016-02-29', '2016-03-01', '2016-03-02', '2016-03-03',
       '2016-03-04', '2016-03-06', '2016-03-07', '2016-03-08',
       '2016-03-09', '2016-03-10', '2016-03-12', '2016-03-14',
       '2016-03-15', '2016-03-16', '2016-03-17', '2016-

In [5]:
orders['order_date'] = orders['order_date'].str.replace('1016', '2016').str.replace('1018', '2018')

In [6]:
orders['order_date'] = pd.to_datetime(orders['order_date'])

In [7]:
orders = orders[(orders['order_date'] >= '2000-01-01') & (orders['order_date'] <= '2024-12-31')]

In [8]:
#check the duplicates
orders.duplicated().sum()

0

In [9]:
#check the nulls
orders.isnull().sum()

order_id                  0
customer_id               0
order_status              0
order_date                0
required_date             0
shipped_date            170
store_id                  0
staff_id                  0
Extraction_Date_Time      0
Source                    0
dtype: int64

###### to fix the nulls in sgipped_date we can the use the avarge between required_date and shipped_date to get the most accurate sgipped_date

In [10]:
#fix the required_date and shipped_date to use them:
orders['required_date'] = orders['required_date'].str.replace('1016', '2016').str.replace('1018', '2018')
orders['required_date'] = pd.to_datetime(orders['required_date'])

In [11]:
orders['shipped_date'] = orders['shipped_date'].str.replace('1016', '2016').str.replace('1018', '2018')
orders['shipped_date'] = pd.to_datetime(orders['shipped_date'])

In [12]:
#to get the correct avg we need to ignore the nulls in shipped_date:
valid_shipped_orders = orders.dropna(subset=['shipped_date'])

In [13]:
average_diff = (valid_shipped_orders['shipped_date'] - valid_shipped_orders['required_date']).mean()
#to only get the days
average_diff_days = average_diff.days
average_diff_days

-1

In [14]:
orders['shipped_date'] = orders.apply(lambda row: row['required_date'] + pd.Timedelta(days=average_diff_days) 
                                      if pd.isnull(row['shipped_date']) else row['shipped_date'], axis=1)
orders.isnull().sum()

order_id                0
customer_id             0
order_status            0
order_date              0
required_date           0
shipped_date            0
store_id                0
staff_id                0
Extraction_Date_Time    0
Source                  0
dtype: int64

In [15]:
file_path = '../Staging_1/orders/orders.csv'

os.makedirs(os.path.dirname(file_path), exist_ok=True)
orders.to_csv(file_path, index = False) 